In [ ]:
!pip install pyomo
! sudo apt-get install glpk-utils --quiet  # GLPK
! sudo apt-get install coinor-cbc --quiet  # CoinOR

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists...
Building dependency tree...
Reading state information...
glpk-utils is already the newest version (4.65-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
coinor-cbc is already the newest version (2.9.9+repack1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
from IPython.display import clear_output

# Installing cbc solver
import matplotlib.pyplot as plt
import numpy as np

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

clear_output()

In [ ]:
from pyomo.environ import *
infinity = float('inf')

model = AbstractModel()
# Covers
model.F = Set()

# Storage
model.N = Set()

# Revenue of each cover
model.c = Param(model.F, within=PositiveReals)
# Size of each cover
model.a = Param(model.F, model.N, within=NonNegativeReals)

# Lower and upper bound on each cover
model.Nmin = Param(model.N, within=NonNegativeReals, default =0.0)
model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)

# Maximum producing per each cover
model.V = Param(model.F, within=PositiveReals)

# Maximum producing of cover consumed
model.Vmax = Param(within=PositiveReals)

# Number of cover
model.x = Var(model.F, within=NonNegativeIntegers)

In [ ]:
# Maximize the revenue of covered
def cost_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.F) + 58500
model.cost = Objective(rule=cost_rule)

# Limit for volume of covers
def volume_rule(model, j):
    value = sum(model.a[i, j] * model.x[i] for i in model.F)
    return inequality(model.Nmin[j], value, model.Nmax[j])

model.volume_limit = Constraint(model.N, rule=volume_rule)

def producing_rule(model):
    return sum(model.V[i]*model.x[i] for i in model.F) <= model.Vmax

model.producing = Constraint(rule=producing_rule)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
!cat LP_cover.py

import pyomo.environ as pyo
infinity = float('inf')

model = pyo.AbstractModel(sense=pyo.maximize)

# Covers
model.F = pyo.Set()
# Storage
model.N = pyo.Set()

# Revenue of each cover
model.c = pyo.Param(model.F, within=pyo.PositiveReals)
# Size of each cover
model.a = pyo.Param(model.F, model.N, within=pyo.NonNegativeReals)

# Lower and upper bound on each cover
model.Nmin = pyo.Param(model.N, within=pyo.NonNegativeReals, default =0.0)
model.Nmax = pyo.Param(model.N, within=pyo.NonNegativeReals, default=infinity)

# Maximum producing per each cover
model.V = pyo.Param(model.F, within=pyo.PositiveReals)
# Maximum producing of cover consumed
model.Vmax = pyo.Param(within=pyo.PositiveReals)

# Number of cover
model.x = pyo.Var(model.F, within=pyo.NonNegativeIntegers)

# Maximize the revenue of covered
def cost_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.F) + 58500
model.cost = pyo.Objective(rule=cost_rule)

# Limit for volume of covers
def volume_rule(model, j):
  

In [ ]:
!cat LP_cover.dat

param:  F:                 c     V  :=
  "Headphones"             5.0    0.0002 
  "Phones"                 7.0    0.00025  
  "Laptops"                12.0   0.0005;

param Vmax := 3.5;

param:  N:       Nmin   Nmax :=
        Size      0      705
        lim1      0      4500
        lim2      0      14000
        lim3      0      4000;

param a:
                             Size   lim1    lim2   lim3 :=
  "Headphones"               0.03    1.0     0.0    0.0    
  "Phones"                   0.05    0.0     1.0    0.0    
  "Laptops"                  0.22    0.0     0.0    0.1    ;


In [ ]:
!pyomo solve --solver=cbc LP_cover.py LP_cover.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.01] Applying solver
[    0.05] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 158400.0
    Solver results file: results.yml
[    0.06] Applying Pyomo postprocessing actions
[    0.06] Pyomo Finished


In [ ]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 158400.0
  Upper bound: 158400.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound:

# Results for y

In [ ]:
!pyomo solve --solver=cbc LP_covery.py LP_covery.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.01] Applying solver
[    0.06] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 158400.0
    Solver results file: results.yml
[    0.06] Applying Pyomo postprocessing actions
[    0.06] Pyomo Finished


In [ ]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 158400.0
  Upper bound: 158400.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 3
  Number of nonzeros: 3
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: